In [ ]:
import geopandas as geop
import pandas as pd
import numpy as np
import warnings

In [ ]:
# Reading crimes csv file

crime_hist = pd.read_csv('MPS LSOA Level Crime (Historical).csv')

In [ ]:
# New data set with 2011 colums and new variable with sum calculation by row. 

crime_2011 = crime_hist[crime_hist.columns[crime_hist.columns.str.startswith(('2011'))]]
crime_2011_totals = crime_2011.sum(axis=1)

In [ ]:
# New data set with 2021 colums and new variable with sum calculation by LSOA (row). 

crime_2021 = crime_hist[crime_hist.columns[crime_hist.columns.str.startswith(('2021'))]]
crime_2021_totals = crime_2021.sum(axis=1)

In [ ]:
# Copy of crime data and merging of 2011 and 2021 totals by row

crime_cases = crime_hist
crime_cases ['2011 Crime Cases'] = crime_2011_totals
crime_cases ['2021 Crime Cases'] = crime_2021_totals

In [ ]:
# Selection of column

crime_cases = crime_cases[['LSOA Code', 'LSOA Name', '2011 Crime Cases', '2021 Crime Cases']]

In [ ]:
# Group by and summary of the dataset, grouping by LSOA Code and Name and them summary of totals by LSOA for 2011 and 2021

crime = crime_cases.groupby(['LSOA Code','LSOA Name']).agg(
    LSOA_Crime2011 =('2011 Crime Cases', 'sum'),
    LSOA_Crime2021 =('2021 Crime Cases', 'sum')
)

crime.head(5)

In [ ]:
# Read in population by place of birth data 2011 and selection of relevant columns

POB_2011 = pd.read_excel('Country of birth.xlsx', sheet_name= '2011')
POB_2011 = POB_2011 [['LSOA code', 'All Usual residents','United Kingdom']]
POB_2011['Non UK Percentage']= 1-(POB_2011['United Kingdom']/POB_2011['All Usual residents'])

POB_2011.drop(columns='United Kingdom', inplace=True)
#POB_2011 = POB_2011 [['LSOA code', 'All Usual residents', 'Non UK Percentage']]
POB_2011 = POB_2011.rename(columns={'LSOA code': 'LSOA Code'})
POB_2011.head(5)

In [ ]:
# Read in population by place of birth data 2021 and selection of relevant columns

POB_2021 = pd.read_excel('Country of birth.xlsx', sheet_name= '2021')
POB_2021 = POB_2021 [['LSOA code', 'All Usual residents','United Kingdom']]
POB_2021['Non UK Percentage']= 1-(POB_2021['United Kingdom']/POB_2021['All Usual residents'])
POB_2021.drop(columns='United Kingdom' ,inplace=True)

POB_2021 = POB_2021.rename(columns={'LSOA code': 'LSOA Code'})
POB_2021.head(5)

In [ ]:
# Find number of null values in POB 2011 and 2022 datasets

# Find mean, count and number of null values for each dataset

# To fix: printing out the fivenum values(i.e. min, 2nd, 3rd, 4th quartile and max)


yearnum = int()

for n in [POB_2011, POB_2021]:
    data_name = [name for name, obj in locals().items() if obj is n][0]
    n['Non UK Percentage'] = pd.Series(n['Non UK Percentage'])
    #Get summary statistics
    #%8.2f
    fivenum_summary = np.percentile(n['Non UK Percentage'].astype(float).dropna(), [0, 25, 50, 75, 100])
    print("{} statistics:".format(data_name) + " Min:{}".format(round(fivenum_summary[0],3)) + ", Q1:{} ".format(round(fivenum_summary[1],3)) + ", Median:{}".format(round(fivenum_summary[2],3))  + ", Q3:{}".format(round(fivenum_summary[3],3)) + ", Max:{}".format(round(fivenum_summary[4],3)))
    print("Mean Non UK population {} ".format(data_name) + " is "+  str(n['All Usual residents'].mean()))
    print("Mean Non UK population percentage for {} ".format(data_name) + " is "+  str(n['Non UK Percentage'].mean()))
    print("Variance Non UK population percentage for {} ".format(data_name) + " is "+  str(n['Non UK Percentage'].var()))
    print("Number of null population percentage values for {} ".format(data_name) + " is "+  str(n['Non UK Percentage'].isnull().sum()))
    print("Count of LSOAs with population percentage values  for {} ".format(data_name) + " is "+  str(n['Non UK Percentage'].count()))


In [ ]:
# Outer join with 2011 population

crime = crime.merge(POB_2011, on='LSOA Code', how='outer')

In [ ]:
# Change of column name

crime = crime.rename(columns={'All Usual residents': '2011 Population'})

In [ ]:
# Outer join with 2021 population

crime = crime.merge(POB_2021, on='LSOA Code', how='outer')

In [ ]:
# Change of column name

crime = crime.rename(columns={'All Usual residents': '2021 Population'})
crime=crime.rename(columns={'Non UK Percentage_x': '2011 Non UK Percentage'})
crime=crime.rename(columns={'Non UK Percentage_y': '2021 Non UK Percentage'})

In [ ]:
crime.head(5)

In [ ]:
# Checking if there are any missing values. There are 6 rows missing crime data. 

crime.isnull().sum()

In [ ]:
# Crime copy for final dataset
crime_final = crime.copy()

In [ ]:
# Calculation of 2011 crime rate by 1000 people

crime_final ['2011 Crime/1000 people'] = crime_final ['LSOA_Crime2011']/ (crime_final['2011 Population']/1000)

In [ ]:
# Calculation of 2021 crime rate by 1000 people

crime_final ['2021 Crime/1000 people'] = crime_final ['LSOA_Crime2021']/ (crime_final['2021 Population']/1000)



In [ ]:
# Reading crime csv file

#crime_hist = pd.read_excel('occupation.xlsx')
# Read multiple sheets
sheet_names = ['2011', '2021', 'change 2011-2021']  # List of sheet names
df_dict = pd.read_excel('occupation.xlsx', sheet_name=sheet_names)

# Access each DataFrame by sheet name
df_2011 = df_dict['2011']
df_2021 = df_dict['2021']
df_change = df_dict['change 2011-2021']


In [ ]:
df_2011.rename(columns={'All usual residents aged 16-74 in employment':'Total Workforce'}, inplace=True)
df_2011.head(5)

In [ ]:
df_2011.isnull().sum()

In [ ]:
df_2021.rename(columns={'All usual residents aged 16 and over in employment':'Total Workforce'}, inplace=True)
df_2021.head(5)

In [ ]:
column_categories = {
    'White Collar': ['1. Managers, directors and senior officials', '2. Professional occupations', '3. Associate professional and technical occupations'],
    'Blue Collar': ['4. Adminis-trative and secretarial occupations', '5. Skilled trades occupations', '8. Process, plant and machine operatives'],
    'Elementary': ['6. Caring, leisure and other service occupations', '7. Sales and customer service occupations', '9. Elementary occupations']
}


In [ ]:
# 2011 data aggreated in 3 groups, update the proportion of each group

df_2011['White Collar Sum'] = df_2011[column_categories['White Collar']].sum(axis=1)
df_2011['White Collar Prop'] = df_2011['White Collar Sum'] / df_2011['Total Workforce']

df_2011['Blue Collar Sum'] = df_2011[column_categories['Blue Collar']].sum(axis=1)
df_2011['Blue Collar Prop'] = df_2011['Blue Collar Sum'] / df_2011['Total Workforce']

df_2011['Elementary Sum'] = df_2011[column_categories['Elementary']].sum(axis=1)
df_2011['Elementary Prop'] = df_2011['Elementary Sum'] / df_2011['Total Workforce']

df_2011 = df_2011.iloc[:, np.r_[0:3, -6:0]]
df_2011.head(5)

In [ ]:
# 2021 data aggreated in 3 groups, update the proportion of each group

df_2021['White Collar Sum'] = df_2021[column_categories['White Collar']].sum(axis=1)
df_2021['White Collar Prop'] = df_2021['White Collar Sum'] / df_2021['Total Workforce']

df_2021['Blue Collar Sum'] = df_2021[column_categories['Blue Collar']].sum(axis=1)
df_2021['Blue Collar Prop'] = df_2021['Blue Collar Sum'] / df_2021['Total Workforce']

df_2021['Elementary Sum'] = df_2021[column_categories['Elementary']].sum(axis=1)
df_2021['Elementary Prop'] = df_2021['Elementary Sum'] / df_2021['Total Workforce']

df_2021 = df_2021.iloc[:, np.r_[2, -6:0]]
df_2021.head(5)

In [ ]:
# 2011 data left joined with 2021 by LSOA code

merged_df = pd.merge(df_2011, df_2021, on='LSOA code', suffixes = (' 2011', ' 2021'), how='left')
merged_df.rename(columns={'LSOA code': 'LSOA Code'}, inplace = True)
merged_df.drop(columns=['local authority code','local authority name'], inplace=True)
merged_df.head(5)

In [ ]:
# Some simple statistics for each group

print("The mean for each group: ")
print(merged_df.iloc[:,3:].mean())
print()
print("The standard deviation for each group: ")
print(merged_df.iloc[:,3:].std())
print()
print("The five numbers for each group: ")
print(merged_df.iloc[:,3:].quantile([0, 0.25, 0.5, 0.75, 1]))

In [ ]:
crime_final = crime_final.merge(merged_df, on = 'LSOA Code', how='outer')
crime_final.head(5)
#Crime_Occupation[Crime_Occupation.isnull().any(axis=1)]

In [ ]:
# Read the crime data of 2021 for City of London
crime_col2021 = pd.read_csv('2021-city-of-london-street.csv')
crime_col2021.head()

In [ ]:
# Select rows of LSOA codes that have NaN values in the Final crime data

na_lsoa = ['E01000001', 'E01000002', 'E01000003', 'E01000005', 'E01032739', 'E01032740']
crime_col2021_filtered = crime_col2021[crime_col2021['LSOA code'].isin(na_lsoa) == True]
crime_col2021_filtered = crime_col2021_filtered.groupby('LSOA code').size().reset_index(name = 'LSOA_Crime2021')
crime_col2021_filtered.rename(columns={'LSOA code': 'LSOA Code'}, inplace=True)
crime_col2021_filtered

In [ ]:
# Now we calculate the crime rate for LSOAs beside the City of London, and one for the City of London alone
crime_met = crime_final[crime_final['LSOA Code'].isin(na_lsoa) == False]
crime_col = crime_final[crime_final['LSOA Code'].isin(na_lsoa) == True]

crime_col['LSOA_Crime2021'] = crime_col2021_filtered['LSOA_Crime2021'].values
crime_col['2021 Crime/1000 people'] = crime_col['LSOA_Crime2021'] / (crime_col['2021 Population']/1000)

crime_2021_met_sum = crime_met['LSOA_Crime2021'].sum()
crime_2021_col_sum = crime_col['LSOA_Crime2021'].sum()

crime_diff = (crime_2021_col_sum - crime_2021_met_sum) / crime_2021_met_sum

print('The total crime cases number in 2021 for the Metropolitan area is', crime_2021_met_sum)
print('The total crime cases number in 2021 for the City of London is', crime_2021_col_sum)
print('The difference in percentage between the two crime rates is', crime_diff*100, '%')

In [ ]:
# Now we calculate the crime rate of City of London in 2011: first calculate the sum by the same ratio to the Metropolitan area in 2021, and then calculate the crime case number of each LSOA by the weight matrix of 2021 crime case number


crime_2011_met_sum = crime_met['LSOA_Crime2011'].sum()
crime_2011_col_sum = round(crime_2011_met_sum * (1 + crime_diff))

crime_col['LSOA_Crime2011'] = round(crime_2011_col_sum * crime_col['LSOA_Crime2021'] / crime_col['LSOA_Crime2021'].sum())
crime_col['2011 Crime/1000 people'] = crime_col['LSOA_Crime2011'] / (crime_col['2011 Population']/1000)
crime_col

In [ ]:
# Now we merge the new data of City of London into the final crime data

crime_final.update(crime_col)
crime_final[crime_final['LSOA Code'].isin(na_lsoa) == True]

In [ ]:
# Final crime data

crime_final.head(5)

In [ ]:
#List of missing values

crime_final[crime_final.isnull().any(axis=1)]


In [ ]:
# Reading 2021 LSOA Boundaries

lsoa_2021 = geop.read_file('LSOA_Boundaries_2021.shp')


In [ ]:
# Change of column name

lsoa_2021 = lsoa_2021.rename(columns={'LSOA21CD':'LSOA Code'})


In [ ]:
# Selection of crime data for 2011 only

crime_2011 = crime_final.copy()
crime_2011 = crime_2011 [['LSOA Code', '2011 Crime/1000 people','2011 Population','2011 Non UK Percentage', 'White Collar Sum 2011','White Collar Prop 2011',	'Blue Collar Sum 2011',	'Blue Collar Prop 2011',	'Elementary Sum 2011',	'Elementary Prop 2011']]

In [ ]:
crime_2011.head()

In [ ]:
# Merging the 2011 crime data with the 2011 LSOA Boundaries.
# Merging the geometry columns based on LSOA Code so that we have a 1-1 mapping between LSOA code, geometry and calculated columns.
# Need to reset the index for LSOA column for later re-use.

CrimeMap_2011 = lsoa_2021.merge(crime_2011)
CrimeMap_2011 = CrimeMap_2011.dissolve('LSOA Code')
CrimeMap_2011.reset_index(inplace=True)

In [ ]:
# View of Merged Data

CrimeMap_2011.head(5)

In [ ]:
# There are only 6 LSOAs with missing crime rate and 37 with a missing immigration rate

CrimeMap_2011.isnull().sum()

In [ ]:
# These are the rows with the missing crime rate values

CrimeMap_2011[CrimeMap_2011.isnull().any(axis=1)]

In [ ]:
# This plot shows that there are some boundaries missing from the data

CrimeMap_2011.plot()

In [ ]:
# Selection of crime data for 2021 only

crime_2021 = crime_final.copy()
crime_2021 = crime_2021 [['LSOA Code', '2021 Crime/1000 people','2021 Population','2021 Non UK Percentage','White Collar Sum 2021','White Collar Prop 2021',	'Blue Collar Sum 2021',	'Blue Collar Prop 2021',	'Elementary Sum 2021',	'Elementary Prop 2021']]
crime_2021.head(5)

In [ ]:
# Merging the 2021 crime data with the 2021 LSOA Boundaries
# Merging the geometry columns based on LSOA Code so that we have a 1-1 mapping between LSOA code, geometry and calculated columns.
# Need to reset the index for LSOA column for later re-use.

CrimeMap_2021 = lsoa_2021.merge(crime_2021)
CrimeMap_2021 = CrimeMap_2021.dissolve('LSOA Code')
CrimeMap_2021.reset_index(inplace=True)

In [ ]:
# View of Merged Data

CrimeMap_2021.head(5)

In [ ]:
# This plot shows that we are not missing any boundaries for 2021

CrimeMap_2021.plot()

In [ ]:
# There are only 6 LSOAs with missing crime rate. LSOA21NMW does not matter to us.

CrimeMap_2021.isnull().sum()

Analysis:-
Analysing the crime rate & relationship with non-UK population the following steps were carried out:-
1. Set inf or NaNs to 0. These were a consequence of a handful (6) LSOAs having no crime rate available in the dataset we used. 
2. Starting with the crime rate & non-UK population rates, as calculated earlier in the code, the overall correlation between the 2 columns,  was found to be 0.197 and 0.243 for 2011 and 2021 respectively.
3. Ranked crime rates & non-UK population rates in descending order, for use in further analysis.
4. Sorted the crime rates in descending order (so LSOA with highest crime rates were first), we found for 2011
    (a)1 LSOA in East London (Newham) followed by Westminister (tourist and business area in Central London) had the highest crime rates.  Looking at crime rank versus non-UK population rank on their own, there's no apparent correlation for Westminister. An implication of this could be that there's other factors here, which would require further data to analyse e.g. being a tourist\business hotspot there's a greater footfall than other LSOAs or non residents could be higher crime targets.  In the case of Newham which has the 11th highest non-UK population rank (& Lewisham 19th) we found in 2011 the population was less than 1/10th that of the LSOA 2011 population mean (1637) which we did a deeper analysis based on doing a weighting for population but this didn't alter the findings. These 2 LSOAs appear to be outliers.
    (b) The LSOA's with the lowest crime rates were also in areas with the lower non-UK population rank. LSOAs in Bexley, Bromley and Kingston upon Thames were the LSOAs with the lowest crime rates for 2011. Once  LSOA population was factored in, a number of Havering and neighbouring LSOAs in Bexley now show as having the lowest crime rates. 

Repeating steps 4 (a) and (b) for 2021 the findings were as follows:-
(a) Westminster LSOAs now feature even more heavily in the top 10 LSOAs with the highest crime rates (up from 4/10 to 7/10), whether we take the pure crime rate or a weighted crime rate factoring in the LSOA population. Newham LSOAs continue to feature in the highest crime rate areas
(b) Areas with the lowest crime rates in 2021 continue to be LSOAs with a low non-UK population rate. This time Croydon has replaced Kingston upon Thames in the lowest crime areas. One interesting callout is in Croydon where there's an LSOA with a high non-UK percentage but a low crime rate which may indicate (deeper analysis required) that crime rates in surrounding LSOAs are a better indicator of crime.


In summary, for both 2011 and 2021 we can see clusters of LSOAs with both highest and lowest crime rates, with the finding that non-UK population isn't significant in areas with high crime rates but is in ares of low crime rates. We also see that neighbouring LSOAs are in similar crime rate categories.

To do:- Draw plots for (a) total crime rates and (b) based on weighted population
Also, look at weighted population formula to get agreement i.e. CrimeMap_2011['2011 Weighted Pop Crime Rate'] = CrimeMap_2011['2011 Crime/1000 people'] * (CrimeMap_2011['2011 Non UK Percentage'] * CrimeMap_2011['2011 Population'])

In [ ]:
pd.set_option('display.width', 600)
print("Number of LSOAs for analysis:" + str(len(CrimeMap_2011)))



CrimeMap_2011_corr = round(CrimeMap_2011['2011 Crime/1000 people'].corr(CrimeMap_2011['2011 Non UK Percentage']),3)

CrimeMap_2011['2011 Weighted Pop Crime Rate'] = CrimeMap_2011['2011 Crime/1000 people'] * (CrimeMap_2011['2011 Non UK Percentage'] * CrimeMap_2011['2011 Population'])

print("Correlation between crime rate and 2011 non-UK population is:{}".format(CrimeMap_2011_corr))


#Setting inf and null as 0. These were due to missing data
CrimeMap_2011_sorted_df = pd.DataFrame(CrimeMap_2011[['2011 Crime/1000 people','2011 Non UK Percentage', '2011 Weighted Pop Crime Rate','2011 Population', 'LSOA Code','LSOA21NM']])
CrimeMap_2011_sorted_df=CrimeMap_2011_sorted_df.replace([np.inf, np.NAN],0)

#sorting by 2011 Non UK Percentage is descending order in order to create a new column ('Non UK Population Rank') which ranks the LSOA by non-UK population percentage for later analysis
CrimeMap_2011_sorted_df = CrimeMap_2011_sorted_df.sort_values(by='2011 Non UK Percentage',ascending=False )
CrimeMap_2011_sorted_df['Non UK Population Rank'] = range(1,len(CrimeMap_2011_sorted_df)+1)


##excluding 0s as we marked inf\nulls as 0. 
CrimeMap_2011_sorted_df_filtered = CrimeMap_2011_sorted_df[CrimeMap_2011_sorted_df['2011 Crime/1000 people'] > 0]
CrimeMap_2011_sorted_df = CrimeMap_2011_sorted_df.sort_values(by='2011 Crime/1000 people',ascending=False )
CrimeMap_2011_sorted_df['Crime Rank'] = range(1,len(CrimeMap_2011_sorted_df)+1)
CrimeMap_2011_sorted_df_filtered = CrimeMap_2011_sorted_df[CrimeMap_2011_sorted_df['2011 Crime/1000 people'] > 0] 


CrimeMap_2011_sorted_df = CrimeMap_2011_sorted_df.sort_values(by='2011 Crime/1000 people',ascending=False )
CrimeMap_2011_sorted_df_filtered = CrimeMap_2011_sorted_df[CrimeMap_2011_sorted_df['2011 Crime/1000 people'] > 0]

print("LSOAs with highest crime rate in 2011:")
print(CrimeMap_2011_sorted_df_filtered.head(10))

print("LSOAs with lowest crime rate in 2011:")
print(CrimeMap_2011_sorted_df_filtered.tail(10))

CrimeMap_2011_sorted_df = CrimeMap_2011_sorted_df.sort_values(by='2011 Weighted Pop Crime Rate',ascending=False )
CrimeMap_2011_sorted_df_filtered = CrimeMap_2011_sorted_df[CrimeMap_2011_sorted_df['2011 Weighted Pop Crime Rate'] > 0]

print("LSOAs with highest crime rate in 2011 weighted by non UK Population:")
print(CrimeMap_2011_sorted_df_filtered.head(10))

print("LSOAs with lowest crime rate in 2011 weighted by non UK Population:")
print(CrimeMap_2011_sorted_df_filtered.tail(10))

In [ ]:
CrimeMap_2021_corr = round(CrimeMap_2021['2021 Crime/1000 people'].corr(CrimeMap_2021['2021 Non UK Percentage']),3)

CrimeMap_2021['2021 Weighted Pop Crime Rate'] = CrimeMap_2021['2021 Crime/1000 people'] * (CrimeMap_2021['2021 Non UK Percentage'] * CrimeMap_2021['2021 Population'])

print("Correlation between crime rate and 2021 non-UK population is:{}".format(CrimeMap_2021_corr))


#Setting inf and null as 0. These were due to missing data
CrimeMap_2021_sorted_df = pd.DataFrame(CrimeMap_2021[['2021 Crime/1000 people','2021 Non UK Percentage', '2021 Weighted Pop Crime Rate','2021 Population', 'LSOA Code','LSOA21NM']])
CrimeMap_2021_sorted_df=CrimeMap_2021_sorted_df.replace([np.inf, np.NAN],0)

#sorting by 2021 Non UK Percentage is descending order in order to create a new column ('Non UK Population Rank') which ranks the LSOA by non-UK population percentage for later analysis
CrimeMap_2021_sorted_df = CrimeMap_2021_sorted_df.sort_values(by='2021 Non UK Percentage',ascending=False )
CrimeMap_2021_sorted_df['Non UK Population Rank'] = range(1,len(CrimeMap_2021_sorted_df)+1)


##excluding 0s as we marked inf\nulls as 0. 
CrimeMap_2021_sorted_df_filtered = CrimeMap_2021_sorted_df[CrimeMap_2021_sorted_df['2021 Crime/1000 people'] > 0]
CrimeMap_2021_sorted_df = CrimeMap_2021_sorted_df.sort_values(by='2021 Crime/1000 people',ascending=False )
CrimeMap_2021_sorted_df['Crime Rank'] = range(1,len(CrimeMap_2021_sorted_df)+1)
CrimeMap_2021_sorted_df_filtered = CrimeMap_2021_sorted_df[CrimeMap_2021_sorted_df['2021 Crime/1000 people'] > 0] 


CrimeMap_2021_sorted_df = CrimeMap_2021_sorted_df.sort_values(by='2021 Crime/1000 people',ascending=False )
CrimeMap_2021_sorted_df_filtered = CrimeMap_2021_sorted_df[CrimeMap_2021_sorted_df['2021 Crime/1000 people'] > 0]

print("LSOAs with highest crime rate in 2021:")
print(CrimeMap_2021_sorted_df_filtered.head(10))

print("LSOAs with lowest crime rate in 2021:")
print(CrimeMap_2021_sorted_df_filtered.tail(10))

CrimeMap_2021_sorted_df = CrimeMap_2021_sorted_df.sort_values(by='2021 Weighted Pop Crime Rate',ascending=False )
CrimeMap_2021_sorted_df_filtered = CrimeMap_2021_sorted_df[CrimeMap_2021_sorted_df['2021 Weighted Pop Crime Rate'] > 0]

print("LSOAs with highest crime rate in 2021 weighted by non UK Population:")
print(CrimeMap_2021_sorted_df_filtered.head(10))

print("LSOAs with lowest crime rate in 2021 weighted by non UK Population:")
print(CrimeMap_2021_sorted_df_filtered.tail(10))